In [3]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: C:\Program Files\Python311\python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd

In [2]:
data2 = pd.read_csv('C:/Users/T.S JAIKISHORE/Downloads/archive (1)/Dataset.csv')
data2.head()

,Hour,HR,O2Sat,Temp,MAP,Resp,BUN,Chloride,Creatinine,Glucose,Hct,Hgb,WBC,Platelets,Age,Gender,HospAdmTime,ICULOS,SepsisLabel,Patient_ID
0,17,76.0,99.0,38.22,98.0,34.0,17.0,105.0,1.3,134.0,40.0,13.8,17.0,122.0,85.27,1,-0.06,20,0,2707
1,16,113.5,97.5,37.60,72.0,21.5,21.0,101.0,1.4,139.0,28.0,9.4,13.6,224.0,76.88,1,-29.98,17,0,20591
2,10,98.0,99.0,38.00,70.0,16.5,8.0,107.0,0.7,163.0,30.2,10.2,10.7,224.0,56.43,0,-6.44,12,0,18402
3,11,107.0,95.0,36.78,65.0,19.0,64.0,102.0,8.7,75.0,36.7,11.8,11.2,222.0,82.11,1,-31.87,12,0,15109
4,25,117.0,97.0,38.00,98.0,31.0,10.0,106.0,0.6,139.0,29.1,9.1,24.1,423.0,41.17,1,-181.52,26,0,79


In [3]:
import pandas as pd
import seaborn as sns 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, roc_auc_score, mean_absolute_error, mean_squared_error
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
import scipy.stats as stats

In [4]:
rows_to_drop_train = data2.loc[data2['Patient_ID'].apply(lambda x: len(str(x)) == 6)]
df_train = data2.drop(rows_to_drop_train.index)
df_train.to_csv('C:/Users/T.S JAIKISHORE/Downloads/archive (1)/data_part3.csv', index=False)

rows_to_drop_test = data2.loc[data2['Patient_ID'].apply(lambda x: len(str(x)) != 6)]
df_test = data2.drop(rows_to_drop_test.index)
df_test.to_csv('C:/Users/T.S JAIKISHORE/Downloads/archive (1)/data_part4.csv', index=False)

In [5]:
df_train = pd.read_csv('C:\\Users\\T.S JAIKISHORE\\Downloads\\archive (1)\\data_part3.csv')
df_test = pd.read_csv('C:\\Users\\T.S JAIKISHORE\\Downloads\\archive (1)\\data_part4.csv')


In [6]:
patients = list(df_test['Patient_ID'].unique())
len(patients)

0

In [7]:
def corr_matrix(df):
    corr = df.corr()
    mask = np.triu(np.ones_like(corr, dtype=bool))
    f, ax = plt.subplots(figsize=(40,40))
    cmap = sns.diverging_palette(220, 10, as_cmap=True)
    sns.heatmap(corr, mask=mask, cmap="Paired", vmax=.3, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [8]:
columns_drop = {'Unnamed: 0','SBP','DBP','EtCO2','BaseExcess', 'HCO3','pH','PaCO2','Alkalinephos', 'Calcium','Magnesium', 
                'Phosphate','Potassium','PTT','Fibrinogen','Unit1','Unit2'}

In [9]:
if 'Unit1' in df_train.columns and 'Unit2' in df_train.columns:
    df_train = df_train.assign(Unit=df_train['Unit1'] + df_train['Unit2'])

df_train_mod = df_train.drop(columns=columns_drop)

KeyError: "['Unnamed: 0', 'pH', 'Phosphate', 'Unit2', 'Calcium', 'DBP', 'Potassium', 'Magnesium', 'Alkalinephos', 'HCO3', 'Fibrinogen', 'PTT', 'PaCO2', 'EtCO2', 'Unit1', 'SBP', 'BaseExcess'] not found in axis"

In [13]:
df_train_impute = df_train_mod.copy()
df_train_impute['Patient_ID'] = df_train['Patient_ID']
columns_impute = list(df_train_impute.columns)

In [14]:
df_train_impute.update(
    df_train_impute.groupby('Patient_ID').transform(lambda x: x.bfill().ffill())
)

In [15]:
null_col = ['TroponinI', 'Bilirubin_direct', 'AST', 'Bilirubin_total', 'Lactate', 'SaO2', 'FiO2', 'Unit']
df_train_impute = df_train_impute.drop(columns=null_col)

In [31]:
def try_gaussian(df, col):
    print('Actual plot')
    
    # Remove NaN and infinite values
    df_clean = df[np.isfinite(df[col])]
    
    # Apply Yeo-Johnson transformation
    try:
        df_clean['col_yj'], param = stats.yeojohnson(df_clean[col])
        print('Yeo-Johnson plot')
    except Exception as e:
        print(f'Yeo-Johnson transformation failed for {col}: {e}')
    
    # Apply Box-Cox transformation if data is positive
    if (df_clean[col] > 0).all():
        try:
            df_clean['col_bc'], param = stats.boxcox(df_clean[col])
            print('Box-Cox plot')

        except Exception as e:
            print(f'Box-Cox transformation failed for {col}: {e}')
    else:
        print(f'Box-Cox transformation skipped for {col} due to non-positive values')
    
    # Apply exponential transformation
    df_clean['col_1.5'] = df_clean[col] ** (1/1.5)
    print('**1/1.5 plot')
    
    df_clean['col_0.5'] = df_clean[col] ** 0.5
    print('**0.5 plot')
    
    # Apply inverse transformation
    df_clean['col_rec'] = 1 / (df_clean[col] + 0.00001)
    print('Inverse plot')
    
    # Apply logarithmic transformation
    df_clean['col_log'] = np.log(df_clean[col] + 1 - df_clean[col].min())
    print('Logarithmic plot')
    
    # Apply square root transformation
    df_clean['col_sqrt'] = np.sqrt(df_clean[col] - df_clean[col].min() + 1)
    print('Square root plot')


In [32]:
lst = ['O2Sat', 'Temp', 'MAP', 'BUN', 'Creatinine', 'Glucose', 'WBC', 'Platelets']
for i in lst:
    print(i)
    try_gaussian(df_train_impute, i)

O2Sat
Actual plot


C:\Users\T.S JAIKISHORE\AppData\Local\Temp\ipykernel_21584\3755568797.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['col_yj'], param = stats.yeojohnson(df_clean[col])


Yeo-Johnson plot


C:\Users\T.S JAIKISHORE\AppData\Local\Temp\ipykernel_21584\3755568797.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['col_bc'], param = stats.boxcox(df_clean[col])
C:\Users\T.S JAIKISHORE\AppData\Local\Temp\ipykernel_21584\3755568797.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['col_1.5'] = df_clean[col] ** (1/1.5)
C:\Users\T.S JAIKISHORE\AppData\Local\Temp\ipykernel_21584\3755568797.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

Box-Cox plot
**1/1.5 plot
**0.5 plot
Inverse plot
Logarithmic plot
Square root plot
Temp
Actual plot


...

C:\Users\T.S JAIKISHORE\AppData\Local\Temp\ipykernel_21584\3755568797.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['col_bc'], param = stats.boxcox(df_clean[col])
C:\Users\T.S JAIKISHORE\AppData\Local\Temp\ipykernel_21584\3755568797.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['col_1.5'] = df_clean[col] ** (1/1.5)
C:\Users\T.S JAIKISHORE\AppData\Local\Temp\ipykernel_21584\3755568797.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

Box-Cox plot
**1/1.5 plot
**0.5 plot
Inverse plot
Logarithmic plot
Square root plot
Platelets
Actual plot


C:\Users\T.S JAIKISHORE\AppData\Local\Temp\ipykernel_21584\3755568797.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['col_yj'], param = stats.yeojohnson(df_clean[col])


Yeo-Johnson plot
Box-Cox plot
**1/1.5 plot
**0.5 plot
Inverse plot
Logarithmic plot
Square root plot


C:\Users\T.S JAIKISHORE\AppData\Local\Temp\ipykernel_21584\3755568797.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['col_bc'], param = stats.boxcox(df_clean[col])
C:\Users\T.S JAIKISHORE\AppData\Local\Temp\ipykernel_21584\3755568797.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['col_1.5'] = df_clean[col] ** (1/1.5)
C:\Users\T.S JAIKISHORE\AppData\Local\Temp\ipykernel_21584\3755568797.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [33]:
df_train_impute_cleaned = df_train_impute.dropna(subset=['SepsisLabel'])
majority_class = df_train_impute_cleaned[df_train_impute_cleaned['SepsisLabel'] == 0]
minority_class = df_train_impute_cleaned[df_train_impute_cleaned['SepsisLabel'] == 1]
majority_class_subset = majority_class.sample(n=2*len(minority_class))
df_train_final = pd.concat([majority_class_subset, minority_class])

In [34]:
df_train_final.head()

,Hour,HR,O2Sat,Temp,MAP,Resp,BUN,Chloride,Creatinine,Glucose,Hct,Hgb,WBC,Platelets,Age,Gender,HospAdmTime,ICULOS,SepsisLabel,Patient_ID
179152,17,76.0,99.0,38.22,98.0,34.0,17.0,105.0,1.3,134.0,40.0,13.8,17.0,122.0,85.27,1,-0.06,20,0,2707
98503,16,113.5,97.5,37.60,72.0,21.5,21.0,101.0,1.4,139.0,28.0,9.4,13.6,224.0,76.88,1,-29.98,17,0,20591
388091,10,98.0,99.0,38.00,70.0,16.5,8.0,107.0,0.7,163.0,30.2,10.2,10.7,224.0,56.43,0,-6.44,12,0,18402
62405,11,107.0,95.0,36.78,65.0,19.0,64.0,102.0,8.7,75.0,36.7,11.8,11.2,222.0,82.11,1,-31.87,12,0,15109
729797,25,117.0,97.0,38.00,98.0,31.0,10.0,106.0,0.6,139.0,29.1,9.1,24.1,423.0,41.17,1,-181.52,26,0,79


In [30]:
df_train.columns

Index(['Unnamed: 0', 'Hour', 'HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP',
       'Resp', 'EtCO2', 'BaseExcess', 'HCO3', 'FiO2', 'pH', 'PaCO2', 'SaO2',
       'AST', 'BUN', 'Alkalinephos', 'Calcium', 'Chloride', 'Creatinine',
       'Bilirubin_direct', 'Glucose', 'Lactate', 'Magnesium', 'Phosphate',
       'Potassium', 'Bilirubin_total', 'TroponinI', 'Hct', 'Hgb', 'PTT', 'WBC',
       'Fibrinogen', 'Platelets', 'Age', 'Gender', 'Unit1', 'Unit2',
       'HospAdmTime', 'ICULOS', 'SepsisLabel', 'Patient_ID', 'Unit'],
      dtype='object')

In [7]:
def find_negative_columns(df):
    negative_columns = [col for col in df.columns if (df[col] < 0).any()]
    return negative_columns

In [8]:
negative_columns = find_negative_columns(df_train_final)

NameError: name 'df_train_final' is not defined

In [38]:
column_to_drop = 'HospAdmTime'  

# Drop the column
df = df_train_final.drop(columns=[column_to_drop])

In [39]:
df_train_final.head()

,Hour,HR,O2Sat,Temp,MAP,Resp,BUN,Chloride,Creatinine,Glucose,Hct,Hgb,WBC,Platelets,Age,Gender,HospAdmTime,ICULOS,SepsisLabel,Patient_ID
179152,17,76.0,99.0,38.22,98.0,34.0,17.0,105.0,1.3,134.0,40.0,13.8,17.0,122.0,85.27,1,-0.06,20,0,2707
98503,16,113.5,97.5,37.60,72.0,21.5,21.0,101.0,1.4,139.0,28.0,9.4,13.6,224.0,76.88,1,-29.98,17,0,20591
388091,10,98.0,99.0,38.00,70.0,16.5,8.0,107.0,0.7,163.0,30.2,10.2,10.7,224.0,56.43,0,-6.44,12,0,18402
62405,11,107.0,95.0,36.78,65.0,19.0,64.0,102.0,8.7,75.0,36.7,11.8,11.2,222.0,82.11,1,-31.87,12,0,15109
729797,25,117.0,97.0,38.00,98.0,31.0,10.0,106.0,0.6,139.0,29.1,9.1,24.1,423.0,41.17,1,-181.52,26,0,79


In [40]:
df_train_final.columns

Index(['Hour', 'HR', 'O2Sat', 'Temp', 'MAP', 'Resp', 'BUN', 'Chloride',
       'Creatinine', 'Glucose', 'Hct', 'Hgb', 'WBC', 'Platelets', 'Age',
       'Gender', 'HospAdmTime', 'ICULOS', 'SepsisLabel', 'Patient_ID'],
      dtype='object')

In [11]:
def calculate_null_percentages(df):
    null_percentages = df.isnull().mean() * 100
    return null_percentages

In [48]:
null_percentages = calculate_null_percentages(df_train_final)

In [49]:
for ele in null_percentages:
    print(ele)

0.0
0.0019452225334578277
0.0427948957360722
1.0076252723311547
0.007780890133831311
0.10893246187363835
2.528789293495176
2.781668222844693
2.727201991907874
2.3187052598817304
2.39067849361967
3.0170401493930905
3.51890756302521
3.4897292250233427
0.0
0.0
0.0
0.0
0.0
0.0


In [50]:
df_train_final.to_csv('C:/Users/T.S JAIKISHORE/Downloads/archive (1)/DataSet.csv', index=False)

In [1]:
import pandas as pd

In [3]:
gg = pd.read_csv('C:/Users/T.S JAIKISHORE/Downloads/archive (1)/DatasetFinal.csv')

In [4]:
gg.head()

,Hour,HR,O2Sat,Temp,MAP,Resp,BUN,Chloride,Creatinine,Glucose,Hct,Hgb,WBC,Platelets,Age,Gender,ICULOS,SepsisLabel,Patient_ID
0,17,76.0,99.0,38.22,98.0,34.0,17.0,105.0,1.3,134.0,40.0,13.8,17.0,122.0,85.27,1,20,0,2707
1,16,113.5,97.5,37.60,72.0,21.5,21.0,101.0,1.4,139.0,28.0,9.4,13.6,224.0,76.88,1,17,0,20591
2,10,98.0,99.0,38.00,70.0,16.5,8.0,107.0,0.7,163.0,30.2,10.2,10.7,224.0,56.43,0,12,0,18402
3,11,107.0,95.0,36.78,65.0,19.0,64.0,102.0,8.7,75.0,36.7,11.8,11.2,222.0,82.11,1,12,0,15109
4,25,117.0,97.0,38.00,98.0,31.0,10.0,106.0,0.6,139.0,29.1,9.1,24.1,423.0,41.17,1,26,0,79


In [5]:
gg.columns

Index(['Hour', 'HR', 'O2Sat', 'Temp', 'MAP', 'Resp', 'BUN', 'Chloride',
       'Creatinine', 'Glucose', 'Hct', 'Hgb', 'WBC', 'Platelets', 'Age',
       'Gender', 'ICULOS', 'SepsisLabel', 'Patient_ID'],
      dtype='object')

In [6]:
def find_required_patient(id):
    patient = gg[gg['Patient_ID'] == id]
    return patient

In [7]:
ids = gg['Patient_ID'].unique()
ids

array([ 2707, 20591, 18402, ..., 14518,  4928,  6257], dtype=int64)

In [3]:
import torch

OSError: [WinError 126] The specified module could not be found. Error loading "C:\Program Files\Python311\Lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

In [153]:
import pandas as pd

In [154]:
sample = pd.read_csv('C:\\Users\\T.S JAIKISHORE\\Downloads\\archive (1)\\DataSetFinal.csv')

In [155]:
sample.head()

,Hour,HR,O2Sat,Temp,MAP,Resp,BUN,Chloride,Creatinine,Glucose,Hct,Hgb,WBC,Platelets,Age,Gender,ICULOS,SepsisLabel,Patient_ID
0,17,76.0,99.0,38.22,98.0,34.0,17.0,105.0,1.3,134.0,40.0,13.8,17.0,122.0,85.27,1,20,0,2707
1,16,113.5,97.5,37.60,72.0,21.5,21.0,101.0,1.4,139.0,28.0,9.4,13.6,224.0,76.88,1,17,0,20591
2,10,98.0,99.0,38.00,70.0,16.5,8.0,107.0,0.7,163.0,30.2,10.2,10.7,224.0,56.43,0,12,0,18402
3,11,107.0,95.0,36.78,65.0,19.0,64.0,102.0,8.7,75.0,36.7,11.8,11.2,222.0,82.11,1,12,0,15109
4,25,117.0,97.0,38.00,98.0,31.0,10.0,106.0,0.6,139.0,29.1,9.1,24.1,423.0,41.17,1,26,0,79


In [156]:
sample['Hour'].max()

335

In [157]:
for i in sample.columns:
    minimum = sample[i].min()
    maximum = sample[i].max()
    print(i,minimum,maximum)

Hour 0 335
HR 20.0 200.0
O2Sat 28.0 100.0
Temp 21.0 41.92
MAP 20.0 298.0
Resp 2.0 67.0
BUN 1.0 234.0
Chloride 38.0 139.0
Creatinine 0.1 17.6
Glucose 14.0 988.0
Hct 9.4 64.6
Hgb 3.2 32.0
WBC 0.1 316.0
Platelets 5.0 1783.0
Age 18.11 89.0
Gender 0 1
ICULOS 1 336
SepsisLabel 0 1
Patient_ID 1 20643


In [158]:
sample = sample.drop('ICULOS', axis=1)

In [159]:
sample.head()

,Hour,HR,O2Sat,Temp,MAP,Resp,BUN,Chloride,Creatinine,Glucose,Hct,Hgb,WBC,Platelets,Age,Gender,SepsisLabel,Patient_ID
0,17,76.0,99.0,38.22,98.0,34.0,17.0,105.0,1.3,134.0,40.0,13.8,17.0,122.0,85.27,1,0,2707
1,16,113.5,97.5,37.60,72.0,21.5,21.0,101.0,1.4,139.0,28.0,9.4,13.6,224.0,76.88,1,0,20591
2,10,98.0,99.0,38.00,70.0,16.5,8.0,107.0,0.7,163.0,30.2,10.2,10.7,224.0,56.43,0,0,18402
3,11,107.0,95.0,36.78,65.0,19.0,64.0,102.0,8.7,75.0,36.7,11.8,11.2,222.0,82.11,1,0,15109
4,25,117.0,97.0,38.00,98.0,31.0,10.0,106.0,0.6,139.0,29.1,9.1,24.1,423.0,41.17,1,0,79


In [160]:
sample.columns

Index(['Hour', 'HR', 'O2Sat', 'Temp', 'MAP', 'Resp', 'BUN', 'Chloride',
       'Creatinine', 'Glucose', 'Hct', 'Hgb', 'WBC', 'Platelets', 'Age',
       'Gender', 'SepsisLabel', 'Patient_ID'],
      dtype='object')

In [161]:
bins = [20.0, 40.0, 60.0, 80.0, 100.0, 120.0, 140.0, 160.0, 180.0, 200.0]
labels = [0,1, 2, 3, 4, 5, 6, 7, 8]

In [162]:
sample['HR'] = pd.cut(sample['HR'], bins=bins, labels=labels, include_lowest=True, right=False)

In [163]:
sample.head()

,Hour,HR,O2Sat,Temp,MAP,Resp,BUN,Chloride,Creatinine,Glucose,Hct,Hgb,WBC,Platelets,Age,Gender,SepsisLabel,Patient_ID
0,17,2,99.0,38.22,98.0,34.0,17.0,105.0,1.3,134.0,40.0,13.8,17.0,122.0,85.27,1,0,2707
1,16,4,97.5,37.60,72.0,21.5,21.0,101.0,1.4,139.0,28.0,9.4,13.6,224.0,76.88,1,0,20591
2,10,3,99.0,38.00,70.0,16.5,8.0,107.0,0.7,163.0,30.2,10.2,10.7,224.0,56.43,0,0,18402
3,11,4,95.0,36.78,65.0,19.0,64.0,102.0,8.7,75.0,36.7,11.8,11.2,222.0,82.11,1,0,15109
4,25,4,97.0,38.00,98.0,31.0,10.0,106.0,0.6,139.0,29.1,9.1,24.1,423.0,41.17,1,0,79


In [164]:
bb = [28.0,38.0,48.0,58.0,68.0,78.0,88.0,98.0,108.0]
gg = [0,1,2,3,4,5,6,7,8]

In [165]:
sample['O2Sat'] = pd.cut(sample['O2Sat'], bins=bins, labels=labels, include_lowest=True, right=False)

In [166]:
sample.head()

,Hour,HR,O2Sat,Temp,MAP,Resp,BUN,Chloride,Creatinine,Glucose,Hct,Hgb,WBC,Platelets,Age,Gender,SepsisLabel,Patient_ID
0,17,2,3,38.22,98.0,34.0,17.0,105.0,1.3,134.0,40.0,13.8,17.0,122.0,85.27,1,0,2707
1,16,4,3,37.60,72.0,21.5,21.0,101.0,1.4,139.0,28.0,9.4,13.6,224.0,76.88,1,0,20591
2,10,3,3,38.00,70.0,16.5,8.0,107.0,0.7,163.0,30.2,10.2,10.7,224.0,56.43,0,0,18402
3,11,4,3,36.78,65.0,19.0,64.0,102.0,8.7,75.0,36.7,11.8,11.2,222.0,82.11,1,0,15109
4,25,4,3,38.00,98.0,31.0,10.0,106.0,0.6,139.0,29.1,9.1,24.1,423.0,41.17,1,0,79


In [167]:
sample['Temp'].dtype

dtype('float64')

In [168]:
sample['Temp'] = pd.to_numeric(sample['Temp'], errors='coerce')


In [169]:
labels = [0,1,2,3,4,5,6]

In [170]:
sample['Temp'].dtype

dtype('float64')

In [171]:
labels

[0, 1, 2, 3, 4, 5, 6]

In [174]:
bins = [21.0, 24.0, 27.0, 30.0, 33.0, 36.0, 39.0, 42.0]
labels = [0, 1, 2, 3, 4, 5, 6]

In [175]:
sample['Temp'] = pd.cut(sample['Temp'], bins=bins, labels=labels, include_lowest=True)

In [177]:
sample.head()

,Hour,HR,O2Sat,Temp,MAP,Resp,BUN,Chloride,Creatinine,Glucose,Hct,Hgb,WBC,Platelets,Age,Gender,SepsisLabel,Patient_ID
0,17,2,3,5,98.0,34.0,17.0,105.0,1.3,134.0,40.0,13.8,17.0,122.0,85.27,1,0,2707
1,16,4,3,5,72.0,21.5,21.0,101.0,1.4,139.0,28.0,9.4,13.6,224.0,76.88,1,0,20591
2,10,3,3,5,70.0,16.5,8.0,107.0,0.7,163.0,30.2,10.2,10.7,224.0,56.43,0,0,18402
3,11,4,3,5,65.0,19.0,64.0,102.0,8.7,75.0,36.7,11.8,11.2,222.0,82.11,1,0,15109
4,25,4,3,5,98.0,31.0,10.0,106.0,0.6,139.0,29.1,9.1,24.1,423.0,41.17,1,0,79


In [178]:
sample.head()

,Hour,HR,O2Sat,Temp,MAP,Resp,BUN,Chloride,Creatinine,Glucose,Hct,Hgb,WBC,Platelets,Age,Gender,SepsisLabel,Patient_ID
0,17,2,3,5,98.0,34.0,17.0,105.0,1.3,134.0,40.0,13.8,17.0,122.0,85.27,1,0,2707
1,16,4,3,5,72.0,21.5,21.0,101.0,1.4,139.0,28.0,9.4,13.6,224.0,76.88,1,0,20591
2,10,3,3,5,70.0,16.5,8.0,107.0,0.7,163.0,30.2,10.2,10.7,224.0,56.43,0,0,18402
3,11,4,3,5,65.0,19.0,64.0,102.0,8.7,75.0,36.7,11.8,11.2,222.0,82.11,1,0,15109
4,25,4,3,5,98.0,31.0,10.0,106.0,0.6,139.0,29.1,9.1,24.1,423.0,41.17,1,0,79


In [179]:
sample['Temp'].min()

0

In [180]:
sample['Temp'].max()

6

In [181]:
#MAP 20.0 298.0
#Resp 2.0 67.0
#BUN 1.0 234.0
#Chloride 38.0 139.0
#Creatinine 0.1 17.6
#Glucose 14.0 988.0
#Hct 9.4 64.6
#Hgb 3.2 32.0
#WBC 0.1 316.0
#Platelets 5.0 

In [182]:
map_bins = [20.0,50.0,80.0,110.0,140.0,170.0,200.0,230.0,260.0,290.0,320.0]
map_labels = [0,1,2,3,4,5,6,7,8,9]

In [183]:
sample['MAP'] = pd.cut(sample['MAP'], bins=map_bins, labels=map_labels, include_lowest=True)

In [184]:
sample.head()

,Hour,HR,O2Sat,Temp,MAP,Resp,BUN,Chloride,Creatinine,Glucose,Hct,Hgb,WBC,Platelets,Age,Gender,SepsisLabel,Patient_ID
0,17,2,3,5,2,34.0,17.0,105.0,1.3,134.0,40.0,13.8,17.0,122.0,85.27,1,0,2707
1,16,4,3,5,1,21.5,21.0,101.0,1.4,139.0,28.0,9.4,13.6,224.0,76.88,1,0,20591
2,10,3,3,5,1,16.5,8.0,107.0,0.7,163.0,30.2,10.2,10.7,224.0,56.43,0,0,18402
3,11,4,3,5,1,19.0,64.0,102.0,8.7,75.0,36.7,11.8,11.2,222.0,82.11,1,0,15109
4,25,4,3,5,2,31.0,10.0,106.0,0.6,139.0,29.1,9.1,24.1,423.0,41.17,1,0,79


In [185]:
resp_bins = [2.0,12.0,22.0,32.0,42.0,52.0,62.0,72.0]
resp_labels = [0,1,2,3,4,5,6]

In [186]:
sample['Resp'] = pd.cut(sample['Resp'],bins=resp_bins, labels=resp_labels, include_lowest=True)

In [187]:
sample.head()

,Hour,HR,O2Sat,Temp,MAP,Resp,BUN,Chloride,Creatinine,Glucose,Hct,Hgb,WBC,Platelets,Age,Gender,SepsisLabel,Patient_ID
0,17,2,3,5,2,3,17.0,105.0,1.3,134.0,40.0,13.8,17.0,122.0,85.27,1,0,2707
1,16,4,3,5,1,1,21.0,101.0,1.4,139.0,28.0,9.4,13.6,224.0,76.88,1,0,20591
2,10,3,3,5,1,1,8.0,107.0,0.7,163.0,30.2,10.2,10.7,224.0,56.43,0,0,18402
3,11,4,3,5,1,1,64.0,102.0,8.7,75.0,36.7,11.8,11.2,222.0,82.11,1,0,15109
4,25,4,3,5,2,2,10.0,106.0,0.6,139.0,29.1,9.1,24.1,423.0,41.17,1,0,79


In [188]:
bun_bins = [1.0,31.0,61.0,91.0,121.0,151.0,181.0,211.0,241.0]
bun_labels = [0,1,2,3,4,5,6,7]

In [189]:
sample['BUN'] = pd.cut(sample['BUN'],bins=bun_bins,labels=bun_labels,include_lowest=True)

In [190]:
sample.head()

,Hour,HR,O2Sat,Temp,MAP,Resp,BUN,Chloride,Creatinine,Glucose,Hct,Hgb,WBC,Platelets,Age,Gender,SepsisLabel,Patient_ID
0,17,2,3,5,2,3,0,105.0,1.3,134.0,40.0,13.8,17.0,122.0,85.27,1,0,2707
1,16,4,3,5,1,1,0,101.0,1.4,139.0,28.0,9.4,13.6,224.0,76.88,1,0,20591
2,10,3,3,5,1,1,0,107.0,0.7,163.0,30.2,10.2,10.7,224.0,56.43,0,0,18402
3,11,4,3,5,1,1,2,102.0,8.7,75.0,36.7,11.8,11.2,222.0,82.11,1,0,15109
4,25,4,3,5,2,2,0,106.0,0.6,139.0,29.1,9.1,24.1,423.0,41.17,1,0,79


In [191]:
chloride_bins = [38.0,58.0,78.0,98.0,118.0,138.0,158.0]
chloride_labels = [0,1,2,3,4,5]

In [192]:
sample['Chloride'] = pd.cut(sample['Chloride'],bins = chloride_bins,labels = chloride_labels,include_lowest=True)

In [193]:
sample.head()

,Hour,HR,O2Sat,Temp,MAP,Resp,BUN,Chloride,Creatinine,Glucose,Hct,Hgb,WBC,Platelets,Age,Gender,SepsisLabel,Patient_ID
0,17,2,3,5,2,3,0,3,1.3,134.0,40.0,13.8,17.0,122.0,85.27,1,0,2707
1,16,4,3,5,1,1,0,3,1.4,139.0,28.0,9.4,13.6,224.0,76.88,1,0,20591
2,10,3,3,5,1,1,0,3,0.7,163.0,30.2,10.2,10.7,224.0,56.43,0,0,18402
3,11,4,3,5,1,1,2,3,8.7,75.0,36.7,11.8,11.2,222.0,82.11,1,0,15109
4,25,4,3,5,2,2,0,3,0.6,139.0,29.1,9.1,24.1,423.0,41.17,1,0,79


In [194]:
sample['Chloride'].min()

0

In [195]:
cr_bins = [0.1,2.1,4.1,6.1,8.1,10.1,12.1,14.1,16.1,18.1]
cr_labels = [0,1,2,3,4,5,6,7,8]

In [196]:
sample['Creatinine'] = pd.cut(sample['Creatinine'],bins = cr_bins,labels = cr_labels,include_lowest=True)

In [197]:
sample.head()

,Hour,HR,O2Sat,Temp,MAP,Resp,BUN,Chloride,Creatinine,Glucose,Hct,Hgb,WBC,Platelets,Age,Gender,SepsisLabel,Patient_ID
0,17,2,3,5,2,3,0,3,0,134.0,40.0,13.8,17.0,122.0,85.27,1,0,2707
1,16,4,3,5,1,1,0,3,0,139.0,28.0,9.4,13.6,224.0,76.88,1,0,20591
2,10,3,3,5,1,1,0,3,0,163.0,30.2,10.2,10.7,224.0,56.43,0,0,18402
3,11,4,3,5,1,1,2,3,4,75.0,36.7,11.8,11.2,222.0,82.11,1,0,15109
4,25,4,3,5,2,2,0,3,0,139.0,29.1,9.1,24.1,423.0,41.17,1,0,79


In [201]:
sample['Glucose'].unique()

array([134.  , 139.  , 163.  ,  75.  , 262.  , 118.  , 101.  , 115.  ,
       103.  , 158.  ,  91.  , 277.  , 147.  , 105.5 ,  60.  ,  93.5 ,
       110.  , 124.  ,  83.  , 111.  , 129.  ,  94.  , 483.  , 132.  ,
       119.  , 106.  , 123.  , 131.  , 140.  , 175.  ,  90.  , 169.  ,
        97.  ,  38.  , 127.  , 109.  , 234.  ,  96.  ,  74.  ,  98.  ,
       152.  ,  80.  ,  62.  , 104.  ,  72.  , 120.  , 136.  , 107.  ,
       167.  , 160.  , 185.  , 187.  , 125.  , 153.  , 117.  ,  92.  ,
       196.  , 125.5 ,  82.  ,  61.  , 108.  , 166.  , 135.  , 232.  ,
       170.  , 102.  , 156.  , 179.  , 100.  , 122.  , 192.  ,  85.  ,
        89.  , 182.  , 126.  , 116.  , 128.  ,  81.  , 133.  ,  79.  ,
       151.  , 121.  , 105.  , 168.  , 213.  ,  69.  , 145.  , 155.  ,
        99.  ,  64.  , 148.  , 142.  , 286.  ,  86.  ,  78.  , 178.  ,
       137.  , 144.  , 141.  , 114.  , 236.  , 198.  , 194.  , 157.  ,
          nan,  84.  , 113.  , 149.  , 150.  , 273.  , 235.  , 321.  ,
      

In [209]:
sample.head()

,Hour,HR,O2Sat,Temp,MAP,Resp,BUN,Chloride,Creatinine,Glucose,Hct,Hgb,WBC,Platelets,Age,Gender,SepsisLabel,Patient_ID
0,17,2,3,5,2,3,0,3,0,NaN,40.0,13.8,17.0,122.0,85.27,1,0,2707
1,16,4,3,5,1,1,0,3,0,NaN,28.0,9.4,13.6,224.0,76.88,1,0,20591
2,10,3,3,5,1,1,0,3,0,NaN,30.2,10.2,10.7,224.0,56.43,0,0,18402
3,11,4,3,5,1,1,2,3,4,NaN,36.7,11.8,11.2,222.0,82.11,1,0,15109
4,25,4,3,5,2,2,0,3,0,NaN,29.1,9.1,24.1,423.0,41.17,1,0,79


In [210]:
km = pd.read_csv('C:\\Users\\T.S JAIKISHORE\\Downloads\\archive (1)\\DataSetFinal.csv')

In [211]:
sample['Glucose'] = km['Glucose']

In [212]:
sample.head()

,Hour,HR,O2Sat,Temp,MAP,Resp,BUN,Chloride,Creatinine,Glucose,Hct,Hgb,WBC,Platelets,Age,Gender,SepsisLabel,Patient_ID
0,17,2,3,5,2,3,0,3,0,134.0,40.0,13.8,17.0,122.0,85.27,1,0,2707
1,16,4,3,5,1,1,0,3,0,139.0,28.0,9.4,13.6,224.0,76.88,1,0,20591
2,10,3,3,5,1,1,0,3,0,163.0,30.2,10.2,10.7,224.0,56.43,0,0,18402
3,11,4,3,5,1,1,2,3,4,75.0,36.7,11.8,11.2,222.0,82.11,1,0,15109
4,25,4,3,5,2,2,0,3,0,139.0,29.1,9.1,24.1,423.0,41.17,1,0,79


In [213]:
glucose_bins = [14.0,114.0,214.0,314.0,414.0,514.0,614.0,714.0,814.0,914.0,1014.0]
glucose_labels = [0,1,2,3,4,5,6,7,8,9]

In [214]:
sample['Glucose'].fillna(10, inplace=True)

C:\Users\T.S JAIKISHORE\AppData\Local\Temp\ipykernel_11916\4242557649.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  sample['Glucose'].fillna(10, inplace=True)


In [215]:
sample.head()

,Hour,HR,O2Sat,Temp,MAP,Resp,BUN,Chloride,Creatinine,Glucose,Hct,Hgb,WBC,Platelets,Age,Gender,SepsisLabel,Patient_ID
0,17,2,3,5,2,3,0,3,0,134.0,40.0,13.8,17.0,122.0,85.27,1,0,2707
1,16,4,3,5,1,1,0,3,0,139.0,28.0,9.4,13.6,224.0,76.88,1,0,20591
2,10,3,3,5,1,1,0,3,0,163.0,30.2,10.2,10.7,224.0,56.43,0,0,18402
3,11,4,3,5,1,1,2,3,4,75.0,36.7,11.8,11.2,222.0,82.11,1,0,15109
4,25,4,3,5,2,2,0,3,0,139.0,29.1,9.1,24.1,423.0,41.17,1,0,79


In [216]:
sample['Glucose'] = pd.cut(sample['Glucose'], bins=glucose_bins, labels=glucose_labels, include_lowest=True)

In [217]:
sample.head()

,Hour,HR,O2Sat,Temp,MAP,Resp,BUN,Chloride,Creatinine,Glucose,Hct,Hgb,WBC,Platelets,Age,Gender,SepsisLabel,Patient_ID
0,17,2,3,5,2,3,0,3,0,1,40.0,13.8,17.0,122.0,85.27,1,0,2707
1,16,4,3,5,1,1,0,3,0,1,28.0,9.4,13.6,224.0,76.88,1,0,20591
2,10,3,3,5,1,1,0,3,0,1,30.2,10.2,10.7,224.0,56.43,0,0,18402
3,11,4,3,5,1,1,2,3,4,0,36.7,11.8,11.2,222.0,82.11,1,0,15109
4,25,4,3,5,2,2,0,3,0,1,29.1,9.1,24.1,423.0,41.17,1,0,79


In [219]:
hct_bins = [9.4,17.4,25.4,33.4,41.4,49.4,57.4,65.4]
hct_labels = [0,1,2,3,4,5,6]

In [222]:
sample['Hct'].isna().sum()

1229

In [223]:
sample['Hct'].fillna(7, inplace=True)

C:\Users\T.S JAIKISHORE\AppData\Local\Temp\ipykernel_11916\699029336.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  sample['Hct'].fillna(7, inplace=True)


In [225]:
sample['Hct'] = pd.cut(sample['Hct'], bins=hct_bins, labels=hct_labels, include_lowest=True)

In [226]:
sample.head()

,Hour,HR,O2Sat,Temp,MAP,Resp,BUN,Chloride,Creatinine,Glucose,Hct,Hgb,WBC,Platelets,Age,Gender,SepsisLabel,Patient_ID
0,17,2,3,5,2,3,0,3,0,1,3,13.8,17.0,122.0,85.27,1,0,2707
1,16,4,3,5,1,1,0,3,0,1,2,9.4,13.6,224.0,76.88,1,0,20591
2,10,3,3,5,1,1,0,3,0,1,2,10.2,10.7,224.0,56.43,0,0,18402
3,11,4,3,5,1,1,2,3,4,0,3,11.8,11.2,222.0,82.11,1,0,15109
4,25,4,3,5,2,2,0,3,0,1,2,9.1,24.1,423.0,41.17,1,0,79


In [227]:
hg_bins = [3.2,7.2,11.2,16.2,20.2,24.2,28.2,32.2]
hg_labels = [0,1,2,3,4,5,6]

In [228]:
sample['Hgb'].isna().sum()

1551

In [230]:
sample['Hgb'].fillna(7,inplace=True)

C:\Users\T.S JAIKISHORE\AppData\Local\Temp\ipykernel_11916\145737778.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  sample['Hgb'].fillna(7,inplace=True)


In [231]:
sample.head()

,Hour,HR,O2Sat,Temp,MAP,Resp,BUN,Chloride,Creatinine,Glucose,Hct,Hgb,WBC,Platelets,Age,Gender,SepsisLabel,Patient_ID
0,17,2,3,5,2,3,0,3,0,1,3,13.8,17.0,122.0,85.27,1,0,2707
1,16,4,3,5,1,1,0,3,0,1,2,9.4,13.6,224.0,76.88,1,0,20591
2,10,3,3,5,1,1,0,3,0,1,2,10.2,10.7,224.0,56.43,0,0,18402
3,11,4,3,5,1,1,2,3,4,0,3,11.8,11.2,222.0,82.11,1,0,15109
4,25,4,3,5,2,2,0,3,0,1,2,9.1,24.1,423.0,41.17,1,0,79


In [232]:
sample['Hgb'] = pd.cut(sample['Hgb'], bins=hg_bins, labels=hg_labels, include_lowest=True)

In [233]:
sample.head()

,Hour,HR,O2Sat,Temp,MAP,Resp,BUN,Chloride,Creatinine,Glucose,Hct,Hgb,WBC,Platelets,Age,Gender,SepsisLabel,Patient_ID
0,17,2,3,5,2,3,0,3,0,1,3,2,17.0,122.0,85.27,1,0,2707
1,16,4,3,5,1,1,0,3,0,1,2,1,13.6,224.0,76.88,1,0,20591
2,10,3,3,5,1,1,0,3,0,1,2,1,10.7,224.0,56.43,0,0,18402
3,11,4,3,5,1,1,2,3,4,0,3,2,11.2,222.0,82.11,1,0,15109
4,25,4,3,5,2,2,0,3,0,1,2,1,24.1,423.0,41.17,1,0,79


In [235]:
#WBC 0.1 316.0
#Platelets 5.0
wbc_bins = [0.1,]

490

In [236]:
start = 0.1
end = 316.0
num_bins = 10

interval_size = (end - start) / num_bins

bins = np.linspace(start, end, num_bins + 1)

labels = [0,1,2,3,4,5,6,7,8,9]

In [242]:
sample['WBC'].fillna(10,inplace=True)

C:\Users\T.S JAIKISHORE\AppData\Local\Temp\ipykernel_11916\1508125225.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  sample['WBC'].fillna(10,inplace=True)


In [243]:
sample['WBC'] = pd.cut(sample['WBC'], bins=bins, labels=labels, include_lowest=True)

In [244]:
sample.head()

,Hour,HR,O2Sat,Temp,MAP,Resp,BUN,Chloride,Creatinine,Glucose,Hct,Hgb,WBC,Platelets,Age,Gender,SepsisLabel,Patient_ID
0,17,2,3,5,2,3,0,3,0,1,3,2,0,122.0,85.27,1,0,2707
1,16,4,3,5,1,1,0,3,0,1,2,1,0,224.0,76.88,1,0,20591
2,10,3,3,5,1,1,0,3,0,1,2,1,0,224.0,56.43,0,0,18402
3,11,4,3,5,1,1,2,3,4,0,3,2,0,222.0,82.11,1,0,15109
4,25,4,3,5,2,2,0,3,0,1,2,1,0,423.0,41.17,1,0,79


In [253]:
sample['Platelets'].min()
sample['Platelets'].max()

1783.0

In [254]:
start = 5.0
end = 1783.0
num_bins = 10

interval_size = (end - start) / num_bins

bins = np.linspace(start, end, num_bins + 1)

labels = [0,1,2,3,4,5,6,7,8,9]

In [255]:
bins

array([   5. ,  182.8,  360.6,  538.4,  716.2,  894. , 1071.8, 1249.6,
       1427.4, 1605.2, 1783. ])

In [257]:
sample['Platelets'].fillna(10,inplace=True)

In [258]:
sample['Platelets'] = pd.cut(sample['Platelets'], bins=bins, labels=labels, include_lowest=True)

In [259]:
sample.head()

,Hour,HR,O2Sat,Temp,MAP,Resp,BUN,Chloride,Creatinine,Glucose,Hct,Hgb,WBC,Platelets,Age,Gender,SepsisLabel,Patient_ID
0,17,2,3,5,2,3,0,3,0,1,3,2,0,0,85.27,1,0,2707
1,16,4,3,5,1,1,0,3,0,1,2,1,0,1,76.88,1,0,20591
2,10,3,3,5,1,1,0,3,0,1,2,1,0,1,56.43,0,0,18402
3,11,4,3,5,1,1,2,3,4,0,3,2,0,1,82.11,1,0,15109
4,25,4,3,5,2,2,0,3,0,1,2,1,0,2,41.17,1,0,79


In [262]:
start = 18.11
end = 89.0
num_bins = 10

interval_size = (end - start) / num_bins

bins = np.linspace(start, end, num_bins + 1)

labels = [0,1,2,3,4,5,6,7,8,9]

In [263]:
bins

array([18.11 , 25.199, 32.288, 39.377, 46.466, 53.555, 60.644, 67.733,
       74.822, 81.911, 89.   ])

In [264]:
sample['Age'].fillna(10,inplace=True)

C:\Users\T.S JAIKISHORE\AppData\Local\Temp\ipykernel_11916\2522307808.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  sample['Age'].fillna(10,inplace=True)


In [265]:
sample['Age'] = pd.cut(sample['Age'], bins=bins, labels=labels, include_lowest=True)

In [266]:
sample.head()

,Hour,HR,O2Sat,Temp,MAP,Resp,BUN,Chloride,Creatinine,Glucose,Hct,Hgb,WBC,Platelets,Age,Gender,SepsisLabel,Patient_ID
0,17,2,3,5,2,3,0,3,0,1,3,2,0,0,9,1,0,2707
1,16,4,3,5,1,1,0,3,0,1,2,1,0,1,8,1,0,20591
2,10,3,3,5,1,1,0,3,0,1,2,1,0,1,5,0,0,18402
3,11,4,3,5,1,1,2,3,4,0,3,2,0,1,9,1,0,15109
4,25,4,3,5,2,2,0,3,0,1,2,1,0,2,3,1,0,79


In [267]:
sample.to_csv('C:\\Users\\T.S JAIKISHORE\\Downloads\\archive (1)\\NormData.csv')

In [1]:
import pandas as pd 
data = pd.read_csv('C:\\Users\\T.S JAIKISHORE\\Downloads\\archive (1)\\NormData.csv')

In [2]:
data.head(n=5)

,Unnamed: 0,Hour,HR,O2Sat,Temp,MAP,Resp,BUN,Chloride,Creatinine,Glucose,Hct,Hgb,WBC,Platelets,Age,Gender,SepsisLabel,Patient_ID
0,0,17,2.0,3.0,5.0,2.0,3.0,0.0,3.0,0.0,1.0,3.0,2,0,0,9,1,0,2707
1,1,16,4.0,3.0,5.0,1.0,1.0,0.0,3.0,0.0,1.0,2.0,1,0,1,8,1,0,20591
2,2,10,3.0,3.0,5.0,1.0,1.0,0.0,3.0,0.0,1.0,2.0,1,0,1,5,0,0,18402
3,3,11,4.0,3.0,5.0,1.0,1.0,2.0,3.0,4.0,0.0,3.0,2,0,1,9,1,0,15109
4,4,25,4.0,3.0,5.0,2.0,2.0,0.0,3.0,0.0,1.0,2.0,1,0,2,3,1,0,79


In [3]:
data.drop('Unnamed: 0',axis=1,inplace=True)

In [4]:
data.head()

,Hour,HR,O2Sat,Temp,MAP,Resp,BUN,Chloride,Creatinine,Glucose,Hct,Hgb,WBC,Platelets,Age,Gender,SepsisLabel,Patient_ID
0,17,2.0,3.0,5.0,2.0,3.0,0.0,3.0,0.0,1.0,3.0,2,0,0,9,1,0,2707
1,16,4.0,3.0,5.0,1.0,1.0,0.0,3.0,0.0,1.0,2.0,1,0,1,8,1,0,20591
2,10,3.0,3.0,5.0,1.0,1.0,0.0,3.0,0.0,1.0,2.0,1,0,1,5,0,0,18402
3,11,4.0,3.0,5.0,1.0,1.0,2.0,3.0,4.0,0.0,3.0,2,0,1,9,1,0,15109
4,25,4.0,3.0,5.0,2.0,2.0,0.0,3.0,0.0,1.0,2.0,1,0,2,3,1,0,79


In [5]:
def replace_nan_with_max_plus_one(array):
    if np.issubdtype(array.dtype, np.number):  # Check if the array is numeric
        max_val = np.nanmax(array)  # Find the maximum value, ignoring nan
        new_value = max_val + 1  # Calculate the new value
        array[np.isnan(array)] = new_value  # Replace nan with new value
    return sorted(array)


In [7]:
import numpy as np
finalDict = {}
exc = ['Hour','SepsisLabel','Patient_ID']
for i in data.columns:
    if i not in exc:
        finalDict[i] = data[i].unique()
        finalDict[i] = replace_nan_with_max_plus_one(finalDict[i])

In [8]:
finalDict

{'HR': [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0],
 'O2Sat': [0.0, 1.0, 2.0, 3.0, 4.0, 5.0],
 'Temp': [0.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0],
 'MAP': [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0],
 'Resp': [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0],
 'BUN': [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0],
 'Chloride': [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0],
 'Creatinine': [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0],
 'Glucose': [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0],
 'Hct': [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0],
 'Hgb': [0, 1, 2, 3, 4, 6],
 'WBC': [0, 1, 2, 3, 4, 5, 6, 7, 9],
 'Platelets': [0, 1, 2, 3, 4, 5, 6, 7, 9],
 'Age': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 'Gender': [0, 1]}

In [9]:
data['SepsisLabel'].count()

51408

In [10]:
sepsis_counts = data['SepsisLabel'].value_counts()

In [11]:
sepsis_counts

SepsisLabel
0    34272
1    17136
Name: count, dtype: int64

In [12]:
data.columns

Index(['Hour', 'HR', 'O2Sat', 'Temp', 'MAP', 'Resp', 'BUN', 'Chloride',
       'Creatinine', 'Glucose', 'Hct', 'Hgb', 'WBC', 'Platelets', 'Age',
       'Gender', 'SepsisLabel', 'Patient_ID'],
      dtype='object')

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
train_data , remaining_data = train_test_split(data,test_size = 0.30,stratify = data['SepsisLabel'],random_state=42)

In [15]:
validation_data , test_data = train_test_split(remaining_data,test_size = 2/3,stratify = remaining_data['SepsisLabel'],random_state=42)

In [16]:
print("Training set distribution:")
print(train_data['SepsisLabel'].value_counts())

print("\nValidation set distribution:")
print(validation_data['SepsisLabel'].value_counts())

print("\nTesting set distribution:")
print(test_data['SepsisLabel'].value_counts())

Training set distribution:
SepsisLabel
0    23990
1    11995
Name: count, dtype: int64

Validation set distribution:
SepsisLabel
0    3427
1    1714
Name: count, dtype: int64

Testing set distribution:
SepsisLabel
0    6855
1    3427
Name: count, dtype: int64


In [17]:
sorted_train_data = train_data.sort_values(by = 'Patient_ID')

In [18]:
sorted_train_data.head(n=10)

,Hour,HR,O2Sat,Temp,MAP,Resp,BUN,Chloride,Creatinine,Glucose,Hct,Hgb,WBC,Platelets,Age,Gender,SepsisLabel,Patient_ID
33443,36,3.0,3.0,5.0,1.0,2.0,0.0,2.0,0.0,1.0,3.0,2,0,1,9,0,0,1
24221,31,4.0,3.0,5.0,2.0,2.0,0.0,2.0,0.0,1.0,3.0,2,0,1,9,0,0,1
20678,32,4.0,3.0,5.0,3.0,2.0,0.0,2.0,0.0,1.0,3.0,2,0,1,9,0,0,1
19478,21,5.0,3.0,5.0,1.0,2.0,2.0,3.0,1.0,2.0,4.0,2,0,0,6,1,0,7
33875,5,5.0,3.0,5.0,1.0,1.0,2.0,3.0,1.0,0.0,4.0,3,0,0,6,1,0,7
33008,19,5.0,3.0,5.0,1.0,1.0,2.0,3.0,1.0,2.0,4.0,2,0,0,6,1,0,7
44395,249,4.0,3.0,5.0,2.0,2.0,0.0,3.0,0.0,1.0,2.0,1,0,4,1,1,1,9
15058,30,3.0,3.0,5.0,1.0,1.0,0.0,3.0,0.0,1.0,3.0,2,0,0,1,1,0,9
44403,257,5.0,3.0,5.0,2.0,2.0,0.0,3.0,0.0,0.0,2.0,1,0,4,1,1,1,9
44401,255,5.0,3.0,5.0,2.0,2.0,0.0,3.0,0.0,1.0,2.0,1,0,4,1,1,1,9


In [19]:
len(sorted_train_data.columns)

18

In [20]:
patient_counts = sorted_train_data['Patient_ID'].value_counts()
single_occurrence_ids = patient_counts[patient_counts == 1].index

In [21]:
import numpy as np
np.random.seed(42) 
ids_to_drop = np.random.choice(single_occurrence_ids, size=len(single_occurrence_ids) // 2, replace=False)


In [22]:
filtered_train_data = sorted_train_data[~sorted_train_data['Patient_ID'].isin(ids_to_drop)]

In [23]:
len(filtered_train_data)

32788

In [25]:
len(sorted_train_data)

35985

In [23]:
filtered_train_data.to_csv('C:\\Users\\T.S JAIKISHORE\\Downloads\\archive (1)\\FilteredTrainData.csv',index=False)

In [24]:
validation_data.to_csv('C:\\Users\\T.S JAIKISHORE\\Downloads\\archive (1)\\ValidationData.csv',index=False)

In [25]:
test_data.to_csv('C:\\Users\\T.S JAIKISHORE\\Downloads\\archive (1)\\TestData.csv',index=False)

In [1]:
import pandas as pd
import numpy as np

In [2]:
train_unique_data = pd.read_csv('C:\\Users\\T.S JAIKISHORE\\Downloads\\archive (1)\\FilteredTrainData.csv')

In [3]:
train_unique_data.head()

,Hour,HR,O2Sat,Temp,MAP,Resp,BUN,Chloride,Creatinine,Glucose,Hct,Hgb,WBC,Platelets,Age,Gender,SepsisLabel,Patient_ID
0,36,3.0,3.0,5.0,1.0,2.0,0.0,2.0,0.0,1.0,3.0,2,0,1,9,0,0,1
1,31,4.0,3.0,5.0,2.0,2.0,0.0,2.0,0.0,1.0,3.0,2,0,1,9,0,0,1
2,32,4.0,3.0,5.0,3.0,2.0,0.0,2.0,0.0,1.0,3.0,2,0,1,9,0,0,1
3,21,5.0,3.0,5.0,1.0,2.0,2.0,3.0,1.0,2.0,4.0,2,0,0,6,1,0,7
4,5,5.0,3.0,5.0,1.0,1.0,2.0,3.0,1.0,0.0,4.0,3,0,0,6,1,0,7


In [4]:
train_unique_data.columns

Index(['Hour', 'HR', 'O2Sat', 'Temp', 'MAP', 'Resp', 'BUN', 'Chloride',
       'Creatinine', 'Glucose', 'Hct', 'Hgb', 'WBC', 'Platelets', 'Age',
       'Gender', 'SepsisLabel', 'Patient_ID'],
      dtype='object')

In [6]:
sepsis_status = train_unique_data.groupby('Patient_ID')['SepsisLabel'].max().reset_index()

In [7]:
sepsis_status['HasSepsis'] = sepsis_status['SepsisLabel']
sepsis_status = sepsis_status.drop(columns='SepsisLabel')

In [8]:
sepsis_status.head()

,Patient_ID,HasSepsis
0,1,0
1,7,0
2,9,1
3,11,1
4,13,0


In [10]:
sepsis_status.to_csv('C:\\Users\\T.S JAIKISHORE\\Downloads\\archive (1)\\TrainSepsisStatus.csv',index=False)

In [11]:
validation_unique_data = pd.read_csv('C:\\Users\\T.S JAIKISHORE\\Downloads\\archive (1)\\ValidationData.csv')

In [12]:
sepsis_status = validation_unique_data.groupby('Patient_ID')['SepsisLabel'].max().reset_index()

In [13]:
sepsis_status['HasSepsis'] = sepsis_status['SepsisLabel']
sepsis_status = sepsis_status.drop(columns='SepsisLabel')

In [14]:
sepsis_status.to_csv('C:\\Users\\T.S JAIKISHORE\\Downloads\\archive (1)\\ValidationSepsisStatus.csv',index=False)

In [15]:
test_unique_data = pd.read_csv('C:\\Users\\T.S JAIKISHORE\\Downloads\\archive (1)\\TestData.csv')   

In [16]:
sepsis_status = test_unique_data.groupby('Patient_ID')['SepsisLabel'].max().reset_index()

In [17]:
sepsis_status['HasSepsis'] = sepsis_status['SepsisLabel']
sepsis_status = sepsis_status.drop(columns='SepsisLabel')

In [18]:
sepsis_status.to_csv('C:\\Users\\T.S JAIKISHORE\\Downloads\\archive (1)\\TestSepsisStatus.csv',index=False)

In [2]:
import pandas as pd
ddd = pd.read_csv('C:\\Users\\T.S JAIKISHORE\\Downloads\\archive (1)\\FilteredTrainData.csv')

In [3]:
nan_counts = ddd.isna().sum()

In [4]:
nan_counts

Hour             0
HR               1
O2Sat           12
Temp           329
MAP              1
Resp            37
BUN            851
Chloride       919
Creatinine     923
Glucose        776
Hct            806
Hgb              0
WBC              0
Platelets        0
Age              0
Gender           0
SepsisLabel      0
Patient_ID       0
dtype: int64

In [5]:
for i in ddd.columns:
    print(i,ddd[i].max(),ddd[i].min())

Hour 335 0
HR 8.0 0.0
O2Sat 4.0 0.0
Temp 6.0 0.0
MAP 9.0 0.0
Resp 6.0 0.0
BUN 7.0 0.0
Chloride 5.0 0.0
Creatinine 8.0 0.0
Glucose 9.0 0.0
Hct 6.0 0.0
Hgb 6 0
WBC 7 0
Platelets 9 0
Age 9 0
Gender 1 0
SepsisLabel 1 0
Patient_ID 20643 1


In [6]:
for i in ddd.columns:
    ddd[i].fillna(ddd[i].max()+1,inplace=True)

C:\Users\T.S JAIKISHORE\AppData\Local\Temp\ipykernel_15272\2443630712.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ddd[i].fillna(ddd[i].max()+1,inplace=True)


In [7]:
nan_counts = ddd.isna().sum()

In [8]:
nan_counts

Hour           0
HR             0
O2Sat          0
Temp           0
MAP            0
Resp           0
BUN            0
Chloride       0
Creatinine     0
Glucose        0
Hct            0
Hgb            0
WBC            0
Platelets      0
Age            0
Gender         0
SepsisLabel    0
Patient_ID     0
dtype: int64

In [9]:
for i in ddd.columns:
    print(i,ddd[i].max(),ddd[i].min())

Hour 335 0
HR 9.0 0.0
O2Sat 5.0 0.0
Temp 7.0 0.0
MAP 10.0 0.0
Resp 7.0 0.0
BUN 8.0 0.0
Chloride 6.0 0.0
Creatinine 9.0 0.0
Glucose 10.0 0.0
Hct 7.0 0.0
Hgb 6 0
WBC 7 0
Platelets 9 0
Age 9 0
Gender 1 0
SepsisLabel 1 0
Patient_ID 20643 1


In [10]:
TrainingData = ddd.to_csv('C:\\Users\\T.S JAIKISHORE\\Downloads\\archive (1)\\TrainingDatabase.csv',index=False)

In [12]:
hhhh = pd.read_csv('C:\\Users\\T.S JAIKISHORE\\Downloads\\archive (1)\\ValidationData.csv')

In [13]:
nan_counts = hhhh.isna().sum()

In [14]:
nan_counts

Hour             0
HR               0
O2Sat            3
Temp            53
MAP              1
Resp             3
BUN            135
Chloride       144
Creatinine     142
Glucose        119
Hct            126
Hgb              0
WBC              0
Platelets        0
Age              0
Gender           0
SepsisLabel      0
Patient_ID       0
dtype: int64

In [15]:
for i in hhhh.columns:
    hhhh[i].fillna(hhhh[i].max()+1,inplace=True)

C:\Users\T.S JAIKISHORE\AppData\Local\Temp\ipykernel_15272\1581811302.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  hhhh[i].fillna(hhhh[i].max()+1,inplace=True)


In [16]:
nan_counts = hhhh.isna().sum()

In [17]:
nan_counts

Hour           0
HR             0
O2Sat          0
Temp           0
MAP            0
Resp           0
BUN            0
Chloride       0
Creatinine     0
Glucose        0
Hct            0
Hgb            0
WBC            0
Platelets      0
Age            0
Gender         0
SepsisLabel    0
Patient_ID     0
dtype: int64

In [18]:
hhhh.to_csv('C:\\Users\\T.S JAIKISHORE\\Downloads\\archive (1)\\ValidationDatabase.csv',index=False)

In [19]:
yyy = pd.read_csv('C:\\Users\\T.S JAIKISHORE\\Downloads\\archive (1)\\TestData.csv')

In [22]:
nan_counts = yyy.isna().sum()

In [23]:
nan_counts

Hour             0
HR               1
O2Sat            6
Temp           106
MAP              2
Resp            13
BUN            264
Chloride       294
Creatinine     285
Glucose        248
Hct            256
Hgb              0
WBC              0
Platelets        0
Age              0
Gender           0
SepsisLabel      0
Patient_ID       0
dtype: int64

In [25]:
for i in yyy.columns:
    yyy[i].fillna(yyy[i].max()+1,inplace=True)

C:\Users\T.S JAIKISHORE\AppData\Local\Temp\ipykernel_15272\204547953.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  yyy[i].fillna(yyy[i].max()+1,inplace=True)


In [26]:
nan_counts = yyy.isna().sum()

In [28]:
nan_counts

Hour           0
HR             0
O2Sat          0
Temp           0
MAP            0
Resp           0
BUN            0
Chloride       0
Creatinine     0
Glucose        0
Hct            0
Hgb            0
WBC            0
Platelets      0
Age            0
Gender         0
SepsisLabel    0
Patient_ID     0
dtype: int64

In [29]:
yyy.to_csv('C:\\Users\\T.S JAIKISHORE\\Downloads\\archive (1)\\TestDatabase.csv',index=False)